In [49]:
!pip install simplejson
!pip install pickle5
!pip install nltk
import pickle5 as pickle
import nltk
import string
import re
import numpy as np
import pandas as pd
import simplejson
import json
#CNN Imports
import os
import sys
import keras
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import model_from_json
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [50]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [39]:
import sys
import os
prefix = "/content/gdrive/My Drive/NLP Assignments/"
sys.path.append(prefix)

In [51]:
json_filename = prefix+"bestmodel.json"
json_file = open(json_filename, "r")
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

In [52]:
model_filename = prefix+"bestmodel.h5"
loaded_model.load_weights(model_filename)
print("Loaded model from disk")

Loaded model from disk


In [53]:
test_filename = prefix+"Test.csv"
test_reviews =pd.read_csv(test_filename)

In [54]:
stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()
MAX_WORDS = 2500
MAX_SEQUENCE_LENGTH = 734

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

test_reviews['clean_text2'] = test_reviews['text'].apply(lambda x: clean_text(x))
X_test, y_test = test_reviews['clean_text2'], test_reviews['label']

# loading tokenizer
token_filename = prefix+"tokenizer.pickle"
with open(token_filename, 'rb') as handle:
    tokenizer = pickle.load(handle)

X_test_seq = tokenizer.texts_to_sequences(X_test)
X_test_data = pad_sequences(X_test_seq, maxlen=MAX_SEQUENCE_LENGTH)
labels_test = to_categorical(np.asarray(y_test))
print('Shape of testing data tensor:', X_test_data.shape)
print('Shape of testing label tensor:', labels_test.shape)

Shape of testing data tensor: (5000, 734)
Shape of testing label tensor: (5000, 2)


In [55]:
loaded_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
score = loaded_model.evaluate(X_test_data, y_test, verbose=0)
print ("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

accuracy: 87.68%


In [57]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# compile the model
loaded_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc',f1_m,precision_m, recall_m])
loss_test, accuracy_test, f1_score_test, precision_test, recall_test = loaded_model.evaluate(X_test_data, y_test, verbose=0)
print('F1 score for test dataset :',f1_score_test*100)

F1 score for test dataset : 87.78820633888245
